# ASMSA: Prepare and check input files

**Next steps**
- [tune.ipynb](tune.ipynb): Perform initial hyperparameter tuning for this molecule
- [train.ipynb](train.ipynb): Use results of previous tuning in more thorough training
- [md.ipynb](md.ipynb): Use a trained model in MD simulation with Gromacs

In [ ]:
import mdtraj as md
import numpy as np
import urllib.request
import asmsa
import os
import re
import tensorflow as tf
import gromacs as gmx
import gromacs.fileformats as gf
import matplotlib.pyplot as plt
import numpy as np

## Prepare input files

Tryptophan cage files are downloaded in this section from our Google drive. 

This is for demonstration purpose, in real use the inputs should be placed here, and _conf, traj, topol, index_ variables set to their filenames names.

In [ ]:
# download trpcage files
urllib.request.urlretrieve("https://drive.google.com/uc?export=download&id=19RZmGgz9goXEAbreUd0OBCKWr5UAVN5d","index_correct.ndx")
urllib.request.urlretrieve("https://drive.google.com/uc?export=download&id=1G-4HRnc1R-LqAArFh0DTY-e5ULd8Goly","topol_correct.top")
urllib.request.urlretrieve("https://drive.google.com/uc?export=download&id=1ddOJPQxXCw3jY3Yds6bm-EwGps8ClVGQ","trpcage_correct.pdb")
urllib.request.urlretrieve("https://drive.google.com/uc?export=download&id=1FRM3-bCdbesShVcyRfk0cj0ILBL1ycbb","trpcage_red.xtc")
urllib.request.urlretrieve("https://drive.google.com/uc?export=download&id=1A28ik84vDhIzyHPHrL8nTxgjzOQAmzLU","trpcage_correct.gro")

In [ ]:
# Define input files

# input conformation
#conf = "alaninedipeptide_H.pdb"
conf = "trpcage_correct.pdb"

# input trajectory
# atom numbering must be consistent with {conf}

#traj = "alaninedipeptide_reduced.xtc"
traj = "trpcage_red.xtc"

# input topology
# expected to be produced with 
#    gmx pdb2gmx -f {conf} -p {topol} -n {index} -o {gro}

# Gromacs changes atom numbering, the index file must be generated and used as well
# gro file is used to generate inverse indexing for plumed.dat

#topol = "topol.top"
topol = "topol_correct.top"
index = 'index_correct.ndx'
gro = 'trpcage_correct.gro'

## Sanity checks

In [ ]:
# Load the trajectory, it should report expected numbers of frames and atoms/residua

tr = md.load(traj,top=conf)
idx=tr[0].top.select("name CA")

# for trivial cases like Ala-Ala, where superposing on CAs fails
#idx=tr[0].top.select("element != H")

tr.superpose(tr[0],atom_indices=idx)

In [ ]:
# Visual check, all frames should look "reasonable"

# Because of different conventions of numbering atoms in proteins,
# PDB file {conf} and the trajectory {traj} can become inconsistent, and this would appear here 
# as rather weird shapes of the molecule

import nglview as nv

v = nv.show_mdtraj(tr)
#v.clear()
#v.add_representation("ribbon")
v

## Split datasets

Split trajectory into 3 parts. Each part will represent training, validation and testing dataset respectively. The workflow is following:
1. Shuffle configurations in trajectory
2. Select proportions to divide the trajectory
3. Divide the trajectory
4. Compute RMSD between
   * **train x validation** trajectory and filter similar structures in train trajectory
   * **train x test** trajectory and filter similar structures in train trajectory
   * **test x validation** trajectory and filter similar structures in test trajectory
5. Transform into internal coordinates
6. Save internal coordinates as datasets which can be loaded in **train.ipynb** and **tune.ipynb** notebooks

In [ ]:
# shuffle the trajectory so the configurations are dispersed across all datasets
np.random.shuffle(tr.xyz)

In [ ]:
# - set proportions for train, validation and test datasets
# - proportions must be equal to 1 when added together
train = .7
validation = .15
test = .15

assert train + validation + test == .9999999999999999 or 1

tr_i = len(tr) * train
X_train = tr.slice(slice(0,int(tr_i)))

va_i = len(tr) * validation
X_validate = tr.slice(slice(int(tr_i),int(tr_i)+int(va_i)))

te_i = len(tr) * test
X_test = tr.slice(slice(int(tr_i)+int(va_i),len(tr)))

X_train.xyz.shape, X_validate.xyz.shape, X_test.xyz.shape

In [ ]:
X_train.save_xtc('train.xtc')
X_validate.save_xtc('validate.xtc')
X_test.save_xtc('test.xtc')

In [ ]:
# eventual recovery
"""
X_train = md.load_xtc('train.xtc',conf)
X_validate = md.load_xtc('validate.xtc',conf)
X_test = md.load_xtc('test.xtc',conf)
"""

In [ ]:
# get RMSD from train trajectory compared to validation trajectory
gmx.select(s=conf,on='backbone.ndx',select='Backbone')
gmx.rms(s=conf,f='train.xtc',f2='validate.xtc',n='backbone.ndx',m='trainxval_rmsd.xpm')

In [ ]:
# load the RMDS matrix
txv = gf.XPM('trainxval_rmsd.xpm')
txv.array.shape

In [ ]:
# minima per row -- for each configuration in train, how far is the nearest one from validation
txv_min = np.min(txv.array,axis=1)
txv_min.shape

In [ ]:
plt.hist(txv_min,bins=50)
plt.show()

In [ ]:
# drop similar structures (to validation trajectory) in train trajectory to avoid dataset being biased
txv_difference = 0.05

train_tr = X_train[np.argwhere(txv_min > txv_difference).flat]
train_tr.xyz.shape

In [ ]:
train_tr.save_xtc('tmp_train.xtc')
#gmx.start(f"rms -f tmp_train.xtc -f2 test.xtc -s {conf} -m trainxtest_rmsd.xpm",wait=True,delete=True,input='4 4')
gmx.rms(s=conf,f='tmp_train.xtc',f2='test.xtc',n='backbone.ndx',m='trainxtest_rmsd.xpm')

In [ ]:
txt = gf.XPM('trainxtest_rmsd.xpm')
txt.array.shape

In [ ]:
txt_min = np.min(txt.array,axis=1)
txt_min.shape

In [ ]:
plt.hist(txt_min,bins=50)
plt.show()

In [ ]:
# ... one more time with test trajectory & test x validation...
txt_difference = 0.05

x_train = train_tr[np.argwhere(txt_min > txt_difference).flat]
x_train.save_xtc('x_train.xtc')

In [ ]:
# test x validation
#gmx.start(f"rms -f test.xtc -f2 validate.xtc -s {conf} -m testxvalidate_rmsd.xpm",wait=True,delete=True,input='4 4')
gmx.rms(f='test.xtc',f2='validate.xtc',s=conf,n='backbone.ndx',m='testxvalidate_rmsd.xpm')

In [ ]:
txv = gf.XPM('testxvalidate_rmsd.xpm')
txv.array.shape

In [ ]:
txv_min = np.min(txv.array,axis=1)
txv_min.shape

In [ ]:
plt.hist(txv_min,bins=50)
plt.show()

In [ ]:
# ... one more time with test trajectory & test x validation...
txv_difference = 0.05

x_test = X_test[np.argwhere(txv_min > txv_difference).flat]
x_test.save_xtc('x_test.xtc')

In [ ]:
# get shapes of filtered trajectories that are to be used as datasets
validate_tr = md.load('validate.xtc', top=conf)

trajs = [x_train, validate_tr, x_test]
x_train.xyz.shape, validate_tr.xyz.shape, x_test.xyz.shape

In [ ]:
# reshuffle the geometries to get frame last so that we can use vectorized calculations later on
geoms = []

for i in range(len(trajs)):
    geoms.append(np.moveaxis(trajs[i].xyz,0,-1))
    print(geoms[i].shape)

In [ ]:
# save geometries

tf.data.Dataset.from_tensor_slices(geoms[0]).shuffle(2048).save('datasets/geoms/train')
tf.data.Dataset.from_tensor_slices(geoms[1]).shuffle(2048).save('datasets/geoms/validate')
tf.data.Dataset.from_tensor_slices(geoms[2]).shuffle(2048).save('datasets/geoms/test')

### Internal coordinates computation

Exercise the ASMSA library on your input. Just check everything appears to work.

There are multiple options that can be combined:
- use traditional internal coordinates (bond distances, angles, and dihedrals) or not
- include additional distances between atoms that may not be bound to express protein folding state more directly
   - dense (all-to-all) atom distances, feasible for very small peptides only
   - sparse atom distances (only some pairs are chosen)
   
**Choose the suitable one in the cell bellow, and copy the same to [tune.ipynb](tune.ipynb) and [train.ipynb](train.ipynb)**, they must be consistent


In [ ]:
# traditional internal coordinates (bond distances, angles, and torsions) only

# mol = asmsa.Molecule(conf,topol)

# internal coordinates and sparse any-any atom distances (not restricted to bonds)
# eventually, top (and index) can be left out to use sparse distances only

# in how many distances each atom should be involved in average
density = 2 # integer in [1, n_atoms-1]

mols = []
for i in range(len(geoms)):
    sparse_dists = asmsa.NBDistancesSparse(geoms[i].shape[0], density=density)
    mols.append(asmsa.Molecule(pdb=conf,top=topol,ndx=index,fms=[sparse_dists]))

# dense distances are feasible for very small (upto 5 residua) peptides only

# dense_dists = asmsa.NBDistancesDense(geom.shape[0])
# mol = asmsa.Molecule(pdb=conf,top=topol,ndx=index,fms=[dense_dists])

In [ ]:
intcoords = []
for i in range(len(mols)):
    intcoords.append(mols[i].intcoord(geoms[i]).T)
    print(intcoords[i].shape)

In [ ]:
[train,validate,test] = intcoords

In [ ]:
# normalize training set
train_mean = np.mean(train,axis=0)
train -= train_mean
train_scale = np.std(train,axis=0)
train /= train_scale

In [ ]:
# normalize test and validation sets
test -= train_mean
test /= train_scale
validate -= train_mean
validate /= train_scale

In [ ]:
# save for usage in tune/train/test phase

tf.data.Dataset.from_tensor_slices(train).save('datasets/intcoords/train')
tf.data.Dataset.from_tensor_slices(validate).save('datasets/intcoords/validate')
tf.data.Dataset.from_tensor_slices(test).save('datasets/intcoords/test')